In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.parent.resolve()))

# Uncomment for more debugging printouts.
"""
import logging
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""
None

In [3]:
from trulens_eval.keys import check_keys
check_keys("OPENAI_API_KEY")

✅ Key OPENAI_API_KEY set from environment (same value found in .env file at /Users/piotrm/Dropbox/repos/github/trulens/.env).


In [ ]:
from langchain.vectorstores.base import VectorStoreRetriever
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from trulens_eval import Tru, TruChain
from trulens_eval import feedback, Feedback
​
from typing import List
import numpy as np
from trulens_eval import Select
​
​
class VectorStoreRetrieverWithScore(VectorStoreRetriever):
    def _get_relevant_documents(
            self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        if self.search_type == "similarity":
            docs_and_scores = self.vectorstore.similarity_search_with_relevance_scores(
                query, **self.search_kwargs
            )
​
            print("From relevant doc in vec store")
            docs = []
            for doc, score in docs_and_scores:
                if score > 0.6:
                    doc.metadata["score"] = score
                    docs.append(doc)
        elif self.search_type == "mmr":
            docs = self.vectorstore.max_marginal_relevance_search(
                query, **self.search_kwargs
            )
        else:
            raise ValueError(f"search_type of {self.search_type} not allowed.")
        return docs
​
​
class FAISSWithScore(FAISS):
    def as_retriever(self) -> VectorStoreRetrieverWithScore:
        return VectorStoreRetrieverWithScore(
            vectorstore=self,
            search_type="similarity",
            search_kwargs={"k": 4},
        )
​
​
class FAISSStore:
    @staticmethod
    def load_vector_store():
        embeddings = OpenAIEmbeddings()#openai_api_key="OPENAI_API_KEY")
        faiss_store = FAISSWithScore.load_local("FAISS_DB_LOC_PATH", embeddings)
        print("Faiss vector DB loaded")
        return faiss_store
​
​
openai = feedback.OpenAI()
​
f_qs_relevance = Feedback(openai.qs_relevance).on_input().on(
    Select.Record.app.combine_docs_chain._call.args.inputs.input_documents[:].page_content
).aggregate(np.min)
​
​
def load_conversational_chain(vector_store):
    llm = ChatOpenAI(
        temperature=0,
        # openai_api_key="OPENAI_API_KEY",
        model_name="gpt-4",
    )
    retriever = vector_store.as_retriever()
    chain = ConversationalRetrievalChain.from_llm(
        llm, retriever, return_source_documents=True
    )
​
    tru = Tru()
​
    truchain = TruChain(
        chain,
        app_id='Support Bot',
        feedbacks=[f_qs_relevance],
        tru=tru,
        with_hugs=False
    )
​
    return truchain
​
​
if __name__=="__main__":
    vector_store = FAISSStore.load_vector_store()
    chain = load_conversational_chain(vector_store)
    chain({"question": "", "chat_history":""})